# Basic CSGO analysis
#### Last Updated: July 27, 2021
The csgo package was developed with easy analysis in mind. To that end, the data parsed goes directly into Pandas DataFrames, as shown in the first example notebook, [Parsing a CSGO demofile](https://github.com/pnxenopoulos/csgo/blob/master/examples/00_Parsing_a_CSGO_demofile.ipynb). To efficiently calculate aggregate statistics from these Pandas Dataframes, the package contains a function `calc_stats`, that filters, groups, and aggregates data based on user input. Furthermore, the package contains eleven functions derived from the function `calc_stats` to calculate popular CSGO aggregate statistics. 

To start, we reference the [demofile](https://www.hltv.org/matches/2349180/gambit-vs-natus-vincere-blast-premier-spring-final-2021) for a match between Gambit and Natus Vincere, where we look at the first map of the series, `de_dust2`.

In [1]:
import operator
from typing import Dict, List, Tuple, Union

import pandas as pd

from csgo.parser import DemoParser

# Create the parser object.
# Set log=True above if you want to produce a logfile for the parser.
demo_parser = DemoParser(demofile = "../../CSGO_Demofiles/gambit-vs-natus-vincere-m1-dust2.dem", demo_id = "GA-NaVi-BLAST2021", parse_rate=128)

# Parse the demofile, output results to a dictionary and a dataframe.
data = demo_parser.parse()
data_df = demo_parser.parse(return_type="df")

17:39:07 [INFO] Go version>=1.14.0
17:39:07 [INFO] Initialized CSGODemoParser with demofile C:\Users\aagrawal-22\CSGO_Demofiles\gambit-vs-natus-vincere-m1-dust2.dem
17:39:07 [INFO] Setting demo id to GA-NaVi-BLAST2021
17:39:07 [INFO] Setting parse rate to 128
17:39:07 [INFO] Running Golang parser from C:\Users\aagrawal-22\Anaconda3\lib\site-packages\csgo-0.1-py3.8.egg\csgo\parser\
17:39:07 [INFO] Looking for file at C:\Users\aagrawal-22\CSGO_Demofiles\gambit-vs-natus-vincere-m1-dust2.dem
17:40:11 [INFO] Wrote demo parse output to GA-NaVi-BLAST2021.json
17:40:11 [INFO] Reading in JSON from GA-NaVi-BLAST2021.json
17:40:12 [INFO] JSON data loaded, available in the `json` attribute to parser
17:40:12 [INFO] Successfully parsed JSON output
17:40:12 [INFO] Successfully returned JSON output
17:40:12 [INFO] Running Golang parser from C:\Users\aagrawal-22\Anaconda3\lib\site-packages\csgo-0.1-py3.8.egg\csgo\parser\
17:40:12 [INFO] Looking for file at C:\Users\aagrawal-22\CSGO_Demofiles\gambit-vs

## CALC_STATS FUNCTION
The function `calc_stats` can be used to calculate aggregate statistics from any of the Pandas DataFrames containing event data. It also allows the user to pass filters for columns. For example, we can use the function to calculate each player's headshot kills in the first half.

In [2]:
# Functions called within calc_stats
def extract_num_filters(filters: Dict[str, Union[List[bool], List[str]]], 
                        key: str) -> Tuple[List[str], List[float]]:
    sign_list = []
    val_list = []
    for index in filters[key]:
        if not isinstance(index, str):
            raise ValueError(f"Filter(s) for column \"{key}\" must be of type " 
                             f"string.")        
        i = 0 
        sign = ""
        while i < len(index) and not index[i].isdecimal(): 
            sign += index[i] 
            end_index = i 
            i += 1
        if sign not in ('==', '!=', '<=', '>=', '<', '>'): 
            raise Exception(f"Invalid logical operator in filters for \"{key}\""
                            f" column.") 
        sign_list.append(sign) 
        try:
            val_list.append(float(index[end_index + 1:])) 
        except ValueError as ve:
            raise Exception(f"Invalid numerical value in filters for \"{key}\" "
                            f"column.") from ve    
    return sign_list, val_list 

def check_filters(df: pd.DataFrame, 
                  filters: Dict[str, Union[List[bool], List[str]]]):
    for key in filters:
        if df.dtypes[key] == "bool":
            for index in filters[key]: 
                if not isinstance(index, bool): 
                    raise ValueError(f"Filter(s) for column \"{key}\" must be " 
                                     f"of type boolean")
        elif df.dtypes[key] == "O":
            for index in filters[key]: 
                if not isinstance(index, str): 
                    raise ValueError(f"Filter(s) for column \"{key}\" must be " 
                                     f"of type string")
        else:
            extract_num_filters(filters, key)  
            
def num_filter_df(df: pd.DataFrame,
                  col: str,
                  sign: str,
                  val: float) -> pd.DataFrame:
    ops = {"==":operator.eq(df[col], val), "!=":operator.ne(df[col], val),
           "<=":operator.le(df[col], val), ">=":operator.ge(df[col], val),
           "<":operator.lt(df[col], val), ">":operator.gt(df[col], val)}
    filtered_df = df.loc[ops[sign]]
    return filtered_df

def filter_df(df: pd.DataFrame,
              filters: Dict[str, Union[List[bool], List[str]]]) -> pd.DataFrame: 
    df_copy = df.copy() 
    check_filters(df_copy, filters) 
    for key in filters:
        if df_copy.dtypes[key] == 'bool' or df_copy.dtypes[key] == 'O': 
            df_copy = df_copy.loc[df_copy[key].isin(filters[key])]
        else:
            i = 0
            for sign in extract_num_filters(filters, key)[0]:
                val = extract_num_filters(filters, key)[1][i]
                df_copy = num_filter_df(df_copy, key, 
                                        extract_num_filters(filters, key)[0][i],
                                        val)
                i += 1
    return df_copy 

In [3]:
def calc_stats(df: pd.DataFrame, 
               filters: Dict[str, Union[List[bool], List[str]]], 
               col_to_groupby: List[str],
               col_to_agg: List[str],
               agg: List[List[str]],    
               col_names: List[str]) -> pd.DataFrame: 
    df_copy = filter_df(df, filters)
    agg_dict = dict(zip(col_to_agg, agg))
    if col_to_agg:
        df_copy = df_copy.groupby(col_to_groupby).agg(agg_dict).reset_index()
    df_copy.columns = col_names
    return df_copy

Below, the data is set to the `Kills` DataFrame, the data is filtered to where the value of the column `IsHeadshot` is True and the value of the column `RoundNum` is less than 16, the data is grouped by `AttackerName`, the column `AttackerName` is aggregated, the aggregation function "size" is used, and the columns are renamed to `Player` and `1st Half HS`.

In [4]:
calc_stats(data_df["Kills"], {"IsHeadshot":[True], "RoundNum":["<=15"]},
           ["AttackerName"], ["AttackerName"], [["size"]], 
           ["Player", "1st Half HS"])

17:41:26 [INFO] NumExpr defaulting to 8 threads.


,Player,1st Half HS
0,Ax1Le,3
1,Boombl4,1
2,Hobbit,6
3,Perfecto,3
4,b1t,5
5,electronic,5
6,interz,5
7,nafany,2
8,s1mple,1


As mentioned earlier, the package contains eleven functions derived from `calc_stats` to efficiently calculate popular CSGO aggregate statistics. The columns to group and aggregate the data by, the aggregation functions, and the column names, do not need to be passed to these functions, only the data and column filters need to be passed.

# KILL_STATS FUNCTION
The function `kill_stats` takes in kill data, round data, kill filters, death filters, and round filters, and returns a DataFrame with kills, deaths, assists, plus-minus, headshots, first kills, kill-death ratio, kills per round, and headshot percentage by player.

In [5]:
def kill_stats(kill_data: pd.DataFrame,
               round_data: pd.DataFrame,
               kill_filters: Dict[str, Union[List[bool], List[str]]] = {},
               death_filters: Dict[str, Union[List[bool], List[str]]] = {},
               round_filters: Dict[str, Union[List[bool], List[str]]] = {}
) -> pd.DataFrame:
    kills = calc_stats(kill_data.loc[kill_data["AttackerTeam"] != 
                                     kill_data["VictimTeam"]],
                       kill_filters, ["AttackerName"], ["AttackerName"], 
                       [["size"]], ["Player", "K"])
    deaths = calc_stats(kill_data, death_filters, ["VictimName"], 
                        ["VictimName"], [["size"]], ["Player", "D"])
    assists = calc_stats(kill_data.loc[(kill_data["AttackerTeam"] != 
                                        kill_data["VictimTeam"]) & 
                                       (kill_data["AssistedFlash"] == False)],
                         kill_filters, ["AssisterName"], ["AssisterName"], 
                         [["size"]], ["Player", "A"])
    first_kills = calc_stats(kill_data.loc[(kill_data["AttackerTeam"] != 
                                            kill_data["VictimTeam"]) &
                                           (kill_data["IsFirstKill"] == True)],
                             kill_filters, ["AttackerName"], ["AttackerName"], 
                             [["size"]], ["Player", "FK"])
    headshots = calc_stats(kill_data.loc[(kill_data["AttackerTeam"] != 
                                          kill_data["VictimTeam"]) & 
                                         (kill_data["IsHeadshot"] == True)], 
                           kill_filters, ["AttackerName"], ["AttackerName"], 
                           [["size"]], ["Player", "HS"])
    headshot_pct = calc_stats(kill_data.loc[kill_data["AttackerTeam"] != 
                                            kill_data["VictimTeam"]], 
                              kill_filters, ["AttackerName"], ["IsHeadshot"], 
                              [["mean"]], ["Player", "HS%"])
    kill_stats = kills.merge(assists, how="outer")
    kill_stats = kill_stats.merge(deaths, how="outer")
    kill_stats["+/-"] = kill_stats["K"] - kill_stats["D"]
    kill_stats["KDR"] = kill_stats["K"] / kill_stats["D"]
    kill_stats["KPR"] = kill_stats["K"] / len(calc_stats(round_data, 
                                                         round_filters, [], [],                                                          
                                                         [], round_data.columns))
    kill_stats = kill_stats.merge(first_kills, how="outer")
    kill_stats = kill_stats.merge(headshots, how="outer")
    kill_stats = kill_stats.merge(headshot_pct, how="outer")
    kill_stats = kill_stats[["Player", "K", "D", "A", "+/-", "HS", "FK", "KDR", 
                             "KPR", "HS%"]]
    kill_stats.fillna(0, inplace=True)
    kill_stats.sort_values(by="K", ascending=False, inplace=True)
    kill_stats.reset_index(drop=True, inplace=True)
    return kill_stats

kill_stats(data_df["Kills"], data_df["Rounds"])

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Hobbit,24,19,3,5,16,5,1.263158,0.800000,0.666667
1,b1t,23,18,1,5,11,4,1.277778,0.766667,0.478261
2,s1mple,22,14,4,8,6,4,1.571429,0.733333,0.272727
3,Ax1Le,18,16,2,2,10,5,1.125000,0.600000,0.555556
4,electronic,18,18,4,0,9,3,1.000000,0.600000,0.500000
5,interz,14,15,1,-1,7,1,0.933333,0.466667,0.500000
6,sh1ro,14,14,5,0,2,3,1.000000,0.466667,0.142857
7,Boombl4,12,16,2,-4,3,1,0.750000,0.400000,0.250000
8,nafany,11,21,3,-10,5,3,0.523810,0.366667,0.454545
9,Perfecto,10,16,2,-6,7,1,0.625000,0.333333,0.700000


# ADR FUNCTION
The function `adr` takes in damage data, round data, damage filters, and round filters, and returns a DataFrame with raw ADR and normalized ADR by player.

In [6]:
def adr(damage_data: pd.DataFrame,
        round_data: pd.DataFrame,
        damage_filters: Dict[str, Union[List[bool], List[str]]] = {},
        round_filters: Dict[str, Union[List[bool], List[str]]] = {}
) -> pd.DataFrame:  
    adr = calc_stats(damage_data.loc[damage_data["AttackerTeam"] != 
                                     damage_data["VictimTeam"]], damage_filters, 
                     ["AttackerName"],["HpDamage", "HpDamageTaken"], [["sum"], 
                                                                      ["sum"]],
                     ["Player", "Raw ADR", "Norm ADR"])
    adr["Raw ADR"] = adr["Raw ADR"] / len(calc_stats(round_data, round_filters, 
                                                     [], [], [], 
                                                     round_data.columns))
    adr["Norm ADR"] = adr["Norm ADR"] / len(calc_stats(round_data, round_filters, 
                                                       [], [], [], 
                                                       round_data.columns))
    adr.fillna(0, inplace=True)
    adr.sort_values(by="Raw ADR", ascending=False, inplace=True)
    adr.reset_index(drop=True, inplace=True)
    return adr

adr(data_df["Damages"], data_df["Rounds"])

,Player,Raw ADR,Norm ADR
0,b1t,138.966667,72.866667
1,Hobbit,119.333333,88.000000
2,s1mple,115.833333,92.700000
3,electronic,85.866667,70.133333
4,sh1ro,76.333333,56.833333
5,Ax1Le,74.566667,58.633333
6,Boombl4,64.333333,57.100000
7,interz,63.733333,54.033333
8,nafany,62.033333,54.000000
9,Perfecto,48.033333,34.133333


# UTIL_DMG FUNCTION
The function `util_dmg` takes in damage data, grenade data, damage filters, and grenade filters, and returns a DataFrame with utility damage, given utility damage, grenades thrown, utility damage per grenade, and given utility damage per grenade by player.

In [7]:
def util_dmg(damage_data: pd.DataFrame,
             grenade_data: pd.DataFrame,
             damage_filters: Dict[str, Union[List[bool], List[str]]] = {},
             grenade_filters: Dict[str, Union[List[bool], List[str]]] = {}
) -> pd.DataFrame:
    util_dmg = calc_stats(damage_data.loc[(damage_data["AttackerTeam"] != 
                                           damage_data["VictimTeam"]) & 
                                          (damage_data["Weapon"].isin([ 
                                               "HE Grenade", 
                                               "Incendiary Grenade", 
                                               "Molotov"
                                          ]))], damage_filters, ["AttackerName"], 
                          ["HpDamage", "HpDamageTaken"], [["sum"], ["sum"]], 
                          ["Player", "UD", "Given UD"])
    nades_thrown = calc_stats(grenade_data.loc[grenade_data["GrenadeType"].isin([
                                                   "HE Grenade", 
                                                   "Incendiary Grenade", 
                                                   "Molotov"
                                               ])], grenade_filters, 
                              ["PlayerName"], ["PlayerName"], [["size"]], 
                              ["Player", "Nades Thrown"])
    util_dmg_stats = util_dmg.merge(nades_thrown, how="outer")
    util_dmg_stats["UD Per Nade"] = (util_dmg_stats["UD"] 
                                     / util_dmg_stats["Nades Thrown"])
    util_dmg_stats["Given UD Per Nade"] = (util_dmg_stats["Given UD"] 
                                           / util_dmg_stats["Nades Thrown"])
    util_dmg_stats.fillna(0, inplace=True)
    util_dmg_stats.sort_values(by="UD", ascending=False, inplace=True)
    util_dmg_stats.reset_index(drop=True, inplace=True)
    return util_dmg_stats

util_dmg(data_df["Damages"], data_df["Grenades"])

,Player,UD,Given UD,Nades Thrown,UD Per Nade,Given UD Per Nade
0,Boombl4,277,277,34,8.147059,8.147059
1,Hobbit,191,191,29,6.586207,6.586207
2,nafany,176,176,25,7.040000,7.040000
3,electronic,111,111,26,4.269231,4.269231
4,sh1ro,105,105,16,6.562500,6.562500
5,s1mple,70,66,10,7.000000,6.600000
6,Perfecto,48,48,35,1.371429,1.371429
7,Ax1Le,45,45,25,1.800000,1.800000
8,b1t,34,34,24,1.416667,1.416667
9,interz,27,27,32,0.843750,0.843750


# FLASH_STATS FUNCTION
The function `flash_stats` takes in flash data, grenade data, flash filters, and grenade filters, and returns a DataFrame with enemies flashed, team flashes, flashes thrown, and enemies flashed per throw by player.

In [8]:
def flash_stats(flash_data: pd.DataFrame,
                grenade_data: pd.DataFrame,
                flash_filters: Dict[str, Union[List[bool], List[str]]] = {},
                grenade_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:  
    enemy_flashes = calc_stats(flash_data.loc[flash_data["AttackerTeam"] != 
                                              flash_data["PlayerTeam"]], 
                               flash_filters, ["AttackerName"], ["AttackerName"], 
                               [["size"]], ["Player", "EF"])
    team_flashes = calc_stats(flash_data.loc[flash_data["AttackerTeam"] == 
                                             flash_data["PlayerTeam"]], 
                              flash_filters, ["AttackerName"], ["AttackerName"], 
                              [["size"]], ["Player", "TF"])
    flashes_thrown = calc_stats(grenade_data.loc[grenade_data["GrenadeType"] == 
                                                 "Flashbang"], flash_filters, 
                                ["PlayerName"], ["PlayerName"], [["size"]], 
                                ["Player", "Flashes Thrown"])
    flash_stats = enemy_flashes.merge(team_flashes, how="outer")
    flash_stats = flash_stats.merge(flashes_thrown, how="outer")
    flash_stats["EF Per Throw"] = flash_stats["EF"] / flash_stats["Flashes Thrown"]
    flash_stats.fillna(0, inplace=True)
    flash_stats.sort_values(by="EF", ascending=False, inplace=True)
    flash_stats.reset_index(drop=True, inplace=True)
    return flash_stats

flash_stats(data_df["Flashes"], data_df["Grenades"])

,Player,EF,TF,Flashes Thrown,EF Per Throw
0,s1mple,55,52,36,1.527778
1,electronic,47,36,28,1.678571
2,b1t,40,35,30,1.333333
3,sh1ro,40,53,36,1.111111
4,Perfecto,36,48,27,1.333333
5,interz,36,37,24,1.500000
6,Ax1Le,35,27,32,1.093750
7,Boombl4,32,47,32,1.000000
8,Hobbit,28,20,23,1.217391
9,nafany,24,16,14,1.714286


# BOMB_STATS FUNCTION
The function `bomb_stats` takes in bomb data, and bomb filters, and returns a DataFrame plants, defuses, and defuse percentage by side.

In [9]:
def bomb_stats(bomb_data: pd.DataFrame,
               bomb_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:  
    team_one = bomb_data["PlayerTeam"].unique()[0]
    team_two = bomb_data["PlayerTeam"].unique()[1]
    team_one_plants = calc_stats(bomb_data.loc[(bomb_data["BombAction"] == 
                                                "plant") & 
                                               (bomb_data["PlayerTeam"] == 
                                                team_one)], bomb_filters, 
                                 ["BombSite"], ["BombSite"], [["size"]], 
                                 ["Bombsite", f"{team_one} Plants"])
    team_two_plants = calc_stats(bomb_data.loc[(bomb_data["BombAction"] == 
                                                "plant") & 
                                               (bomb_data["PlayerTeam"] == 
                                                team_two)], bomb_filters, 
                                 ["BombSite"], ["BombSite"], [["size"]],
                                 ["Bombsite", f"{team_two} Plants"])
    team_one_defuses = calc_stats(bomb_data.loc[(bomb_data["BombAction"] == 
                                                 "defuse") & 
                                                (bomb_data["PlayerTeam"] == 
                                                 team_one)], bomb_filters, 
                                  ["BombSite"], ["BombSite"], [["size"]],
                                  ["Bombsite", f"{team_one} Defuses"])
    team_two_defuses = calc_stats(bomb_data.loc[(bomb_data["BombAction"] == 
                                                 "defuse") & 
                                                (bomb_data["PlayerTeam"] == 
                                                 team_two)], bomb_filters, 
                                  ["BombSite"], ["BombSite"], [["size"]],
                                  ["Bombsite", f"{team_two} Defuses"])
    bomb_stats = team_one_plants.merge(team_two_defuses, how="outer")
    bomb_stats[f"{team_two} Defuse %"] = (bomb_stats[f"{team_two} Defuses"] 
                                         / bomb_stats[f"{team_one} Plants"])
    bomb_stats = bomb_stats.merge(team_two_plants, how="outer")
    bomb_stats = bomb_stats.merge(team_one_defuses, how="outer")
    bomb_stats[f"{team_one} Defuse %"] = (bomb_stats[f"{team_one} Defuses"] 
                                         / bomb_stats[f"{team_two} Plants"])
    bomb_stats.loc[2]=["A and B", bomb_stats[f"{team_one} Plants"].sum(), 
                       bomb_stats[f"{team_two} Defuses"].sum(),
                       (bomb_stats[f"{team_two} Defuses"].sum() / 
                        bomb_stats[f"{team_one} Plants"].sum()), 
                       bomb_stats[f"{team_two} Plants"].sum(), 
                       bomb_stats[f"{team_one} Defuses"].sum(),
                       (bomb_stats[f"{team_one} Defuses"].sum() / 
                        bomb_stats[f"{team_two} Plants"].sum())] 
    bomb_stats.fillna(0, inplace=True)
    return bomb_stats

bomb_stats(data_df["BombEvents"])

,Bombsite,Gambit Plants,Natus Vincere Defuses,Natus Vincere Defuse %,Natus Vincere Plants,Gambit Defuses,Gambit Defuse %
0,A,6,1,0.166667,6,3,0.500000
1,B,3,1,0.333333,3,1,0.333333
2,A and B,9,2,0.222222,9,4,0.444444


# ECON_STATS FUNCTION
The function `econ_stats` takes in round data and returns a DataFrame with buy type, average equipment value, and average spend by side.

In [10]:
def econ_stats(round_data: pd.DataFrame,
               round_data_json: List[Dict]) -> pd.DataFrame:  
    team_one = round_data_json[0]["TTeam"]
    team_two = round_data_json[0]["CTTeam"]
    team_one_T_val = 0
    team_one_CT_val = 0
    team_two_T_val = 0
    team_two_CT_val = 0
    team_one_T_spend = 0
    team_one_CT_spend = 0
    team_two_T_spend = 0
    team_two_CT_spend = 0
    first_half = 0
    second_half = 0
    team_one_T_buy = calc_stats(round_data.loc[round_data["RoundNum"] <= 15], 
                                {}, ["TBuyType"], ["TBuyType"], [["size"]], 
                                ["Side", f"{team_one} T"])
    team_one_CT_buy = calc_stats(round_data.loc[round_data["RoundNum"] > 15], {}, 
                                 ["CTBuyType"], ["CTBuyType"], [["size"]], 
                                 ["Side", f"{team_one} CT"])
    team_two_T_buy = calc_stats(round_data.loc[round_data["RoundNum"] > 15], {}, 
                                ["TBuyType"], ["TBuyType"], [["size"]], 
                                ["Side", f"{team_two} T"])
    team_two_CT_buy = calc_stats(round_data.loc[round_data["RoundNum"] < 15], {}, 
                                 ["CTBuyType"], ["CTBuyType"], [["size"]], 
                                 ["Side", f"{team_two} CT"])
    for rd in round_data_json:
        if rd["RoundNum"] <= 15:
            team_one_T_val += rd["TStartEqVal"]
            team_two_CT_val += rd["CTStartEqVal"]
            team_one_T_spend += rd["TSpend"]
            team_two_CT_spend += rd["CTSpend"]
            first_half += 1
        else:
            team_one_CT_val += rd["CTStartEqVal"]
            team_two_T_val += rd["TStartEqVal"]
            team_one_CT_spend += rd["CTSpend"]
            team_two_T_spend += rd["TSpend"]
            second_half += 1
    if first_half == 0: first_half = 1
    if second_half == 0: second_half = 1
    team_one_T_val = team_one_T_val / first_half
    team_one_CT_val = team_one_CT_val / first_half
    team_two_T_val= team_two_T_val / second_half
    team_two_CT_val = team_two_CT_val / second_half
    team_one_T_spend = team_one_T_spend / first_half
    team_one_CT_spend = team_one_CT_spend / first_half
    team_two_T_spend = team_two_T_spend / second_half
    team_two_CT_spend = team_two_CT_spend / second_half
    econ_stats = team_one_T_buy.merge(team_one_CT_buy, how="outer")    
    econ_stats = econ_stats.merge(team_two_T_buy, how="outer")    
    econ_stats = econ_stats.merge(team_two_CT_buy, how="outer") 
    econ_stats.loc[len(econ_stats)] = ["Avg EQ Value", team_one_T_val, 
                                       team_two_CT_val, team_two_T_val, 
                                       team_one_CT_val]
    econ_stats.loc[len(econ_stats)] = ["Avg Spend", team_one_T_spend, 
                                       team_two_CT_spend, team_two_T_spend, 
                                       team_one_CT_spend]
    econ_stats.fillna(0, inplace=True)
    econ_stats.iloc[:, 1:] = econ_stats.iloc[:, 1:].astype(int)
    econ_stats = econ_stats.transpose()
    econ_stats.reset_index(inplace=True)
    econ_stats.columns = econ_stats.iloc[0]
    econ_stats.drop(0, inplace=True)
    econ_stats.reset_index(drop=True, inplace=True)
    return econ_stats

econ_stats(data_df["Rounds"], data["GameRounds"])

,Side,Eco,Full Buy,Half Buy,Pistol,Full Eco,Avg EQ Value,Avg Spend
0,Gambit T,1,12,1,1,0,21023,12946
1,Gambit CT,0,13,0,1,1,27386,12713
2,Natus Vincere T,0,11,3,1,0,22133,13250
3,Natus Vincere CT,1,11,1,1,0,24183,13403


# KILL_BREAKDOWN FUNCTION
The function `kill_breakdown` takes in kill data and kill filters, and returns a DataFrame with kills by weapon type by player.

In [11]:
def weapon_type(weapon: str) -> str:
    if weapon in ["Knife"]:
        return "Melee Kills"
    elif weapon in ["CZ-75 Auto", "Desert Eagle", "Dual Berettas", "Five-SeveN",
                    "Glock-18", "P2000", "P250", "R8 Revolver", "Tec-9", 
                    "USP-S"]:
        return "Pistol Kills"
    elif weapon in ["MAG-7", "Nova", "Sawed-Off", "XM1014"]:
        return "Shotgun Kills"
    elif weapon in ["MAC-10", "MP5-SD", "MP7", "MP9", "P90", "PP-Bizon",
                    "UMP-45"]:
        return "SMG Kills"
    elif weapon in ["AK-47", "AUG", "FAMAS", "Galil AR", "M4A1-S", "M4A4",
                    "SG 553"]:
        return "Assault Rifle Kills"
    elif weapon in ["M249", "Negev"]:
        return "Machine Gun Kills"
    elif weapon in ["AWP", "G3SG1", "SCAR-20", "SSG 08"]:
        return "Sniper Rifle Kills"
    else:
        return "Utility Kills"
    
    
def kill_breakdown(kill_data: pd.DataFrame,
                   kill_filters: Dict[str, Union[List[bool], List[str]]] = {}
) -> pd.DataFrame:
    kill_breakdown = kill_data.loc[kill_data["AttackerTeam"] != 
                                   kill_data["VictimTeam"]].copy()
    kill_breakdown["Kills Type"] = kill_breakdown.apply(lambda row: weapon_type(
                                                            row["Weapon"]), 
                                                        axis=1)
    kill_breakdown = calc_stats(kill_breakdown, kill_filters, ["AttackerName", 
                                                               "Kills Type"],
                                ["AttackerName"], [["size"]], [
                                                               "Player", 
                                                               "Kills Type", 
                                                               "Kills"
                                                              ])
    kill_breakdown = kill_breakdown.pivot(index="Player", columns="Kills Type",
                                         values="Kills")
    for col in ["Melee Kills", "Pistol Kills", "Shotgun Kills", "SMG Kills", 
                "Assault Rifle Kills", "Machine Gun Kills", "Sniper Rifle Kills", 
                "Utility Kills"]:
        if not col in kill_breakdown.columns:
            kill_breakdown.insert(0, col, 0)
        kill_breakdown[col].fillna(0, inplace=True)
        kill_breakdown[col] = kill_breakdown[col].astype(int)
    kill_breakdown["Total Kills"] = kill_breakdown.iloc[0:].sum(axis=1)
    kill_breakdown.reset_index(inplace=True)
    kill_breakdown = kill_breakdown.rename_axis(None, axis=1)
    kill_breakdown = kill_breakdown[["Player", "Melee Kills", "Pistol Kills",
                                     "Shotgun Kills", "SMG Kills", 
                                     "Assault Rifle Kills", "Machine Gun Kills",
                                     "Sniper Rifle Kills", "Utility Kills", 
                                     "Total Kills"]]
    kill_breakdown.sort_values(by="Total Kills", ascending=False, inplace=True)
    kill_breakdown.reset_index(drop=True, inplace=True)
    return kill_breakdown

kill_breakdown(data_df["Kills"])

,Player,Melee Kills,Pistol Kills,Shotgun Kills,SMG Kills,Assault Rifle Kills,Machine Gun Kills,Sniper Rifle Kills,Utility Kills,Total Kills
0,Hobbit,0,4,0,1,19,0,0,0,24
1,b1t,0,2,0,1,8,0,12,0,23
2,s1mple,0,3,0,0,5,0,13,1,22
3,Ax1Le,0,4,0,1,13,0,0,0,18
4,electronic,0,4,0,0,14,0,0,0,18
5,interz,0,0,0,2,12,0,0,0,14
6,sh1ro,0,1,0,0,0,0,11,2,14
7,Boombl4,0,0,0,0,10,0,2,0,12
8,nafany,0,4,0,0,7,0,0,0,11
9,Perfecto,0,2,0,0,8,0,0,0,10


# UTIL_DMG_BREAKDOWN FUNCTION
The function `util_dmg_breakdown` takes in damage data, grenade data, damage filters, and grenade filters, and returns a DataFrame with utility damage, given utility damage, grenades thrown, utility damage per grenade, and given utility damage per grenade by grenade type by player.

In [12]:
def util_dmg_breakdown(damage_data: pd.DataFrame,
                       grenade_data: pd.DataFrame,
                       damage_filters: Dict[str, Union[List[bool], 
                                                       List[str]]] = {},
                       grenade_filters: Dict[str, Union[List[bool], 
                                                        List[str]]] = {}
) -> pd.DataFrame: 
    util_dmg = calc_stats(damage_data.loc[(damage_data["AttackerTeam"] != 
                                           damage_data["VictimTeam"]) & 
                                          (damage_data["Weapon"].isin([
                                               "HE Grenade", 
                                               "Incendiary Grenade",
                                               "Molotov"
                                          ]))], damage_filters, ["AttackerName", 
                                                                 "Weapon"], 
                          ["HpDamage", "HpDamageTaken"], [["sum"], ["sum"]], 
                          ["Player", "Nade Type", "UD", "Given UD"])
    nades_thrown = calc_stats(grenade_data.loc[grenade_data["GrenadeType"].isin([
                                                   "HE Grenade", 
                                                   "Incendiary Grenade", 
                                                   "Molotov"
                                               ])], grenade_filters, 
                              ["PlayerName", "GrenadeType"], ["PlayerName"], 
                              [["size"]], ["Player", "Nade Type","Nades Thrown"])
    util_dmg_breakdown = util_dmg.merge(nades_thrown, how="outer", on = 
                                        ["Player", "Nade Type"])
    util_dmg_breakdown["UD Per Nade"] = (util_dmg_breakdown["UD"] 
                                         / util_dmg_breakdown["Nades Thrown"])
    util_dmg_breakdown["Given UD Per Nade"] = (util_dmg_breakdown["Given UD"]
                                               / util_dmg_breakdown["Nades Thrown"])
    util_dmg_breakdown.fillna(0, inplace=True)
    util_dmg_breakdown.sort_values(by=["Player", "UD"], ascending=[True, False], 
                                   inplace=True)
    util_dmg_breakdown.reset_index(drop=True, inplace=True)
    return util_dmg_breakdown

util_dmg_breakdown(data_df["Damages"], data_df["Grenades"])

,Player,Nade Type,UD,Given UD,Nades Thrown,UD Per Nade,Given UD Per Nade
0,Ax1Le,HE Grenade,26.0,26.0,9,2.888889,2.888889
1,Ax1Le,Molotov,19.0,19.0,6,3.166667,3.166667
2,Ax1Le,Incendiary Grenade,0.0,0.0,10,0.000000,0.000000
3,Boombl4,HE Grenade,213.0,213.0,13,16.384615,16.384615
4,Boombl4,Incendiary Grenade,40.0,40.0,11,3.636364,3.636364
5,Boombl4,Molotov,24.0,24.0,10,2.400000,2.400000
6,Hobbit,HE Grenade,167.0,167.0,10,16.700000,16.700000
7,Hobbit,Molotov,24.0,24.0,9,2.666667,2.666667
8,Hobbit,Incendiary Grenade,0.0,0.0,10,0.000000,0.000000
9,Perfecto,HE Grenade,48.0,48.0,15,3.200000,3.200000


# WIN_BREAKDOWN FUNCTION
The function `win_breakdown` takes in round data returns a DataFrame with win type by team.

In [13]:
def win_breakdown(round_data: pd.DataFrame,
                  round_filters: Dict[str, Union[List[bool], List[str]]] = {}
) -> pd.DataFrame:     
    win_breakdown = calc_stats(round_data, round_filters, ["WinningTeam", 
                                                           "RoundEndReason"],
                               ["RoundEndReason"], [["size"]], [
                                                                "Team", 
                                                                "RoundEndReason", 
                                                                "Count"
                               ])
    win_breakdown = win_breakdown.pivot(index="Team", columns="RoundEndReason", 
                                        values="Count")
    win_breakdown.reset_index(inplace=True)
    win_breakdown = win_breakdown.rename_axis(None, axis=1)
    win_breakdown["Total"] = win_breakdown.iloc[0:].sum(axis=1)
    return win_breakdown

win_breakdown(data_df["Rounds"])

,Team,BombDefused,CTWin,TargetBombed,TargetSaved,TerroristsWin,Total
0,Gambit,4,1,4,4,3,16
1,Natus Vincere,2,2,4,4,2,14


# PLAYER_BOX_SCORE_FUNCTION
The function `player_box_score` takes in damage data, flash data, grenade data, kill data, round data, and filters for each group of data, and return a player box score DataFrame containing statistics from each group of data.

In [14]:
def player_box_score(damage_data: pd.DataFrame,
                     flash_data: pd.DataFrame,
                     grenade_data: pd.DataFrame,
                     kill_data: pd.DataFrame,
                     round_data: pd.DataFrame,
                     damage_filters: Dict[str, Union[List[bool], 
                                                     List[str]]] = {},
                     flash_filters: Dict[str, Union[List[bool], 
                                                    List[str]]] = {},
                     grenade_filters: Dict[str, Union[List[bool], 
                                                      List[str]]] = {},
                     kill_filters: Dict[str, Union[List[bool], List[str]]] = {},
                     death_filters: Dict[str, Union[List[bool], 
                                                    List[str]]] = {},
                     round_filters: Dict[str, Union[List[bool], List[str]]] = {}
) -> pd.DataFrame:
    kills_df = kill_stats(kill_data, round_data, kill_filters, 
                          death_filters, round_filters)
    kills_df = kills_df[["Player", "K", "D", "A", "+/-", "KDR", "HS%"]]
    adr_df = adr(damage_data, round_data, damage_filters, round_filters)
    adr_df = adr_df[["Player", "Norm ADR"]]
    adr_df.columns = ["Player", "ADR"]
    ud_df = util_dmg(damage_data, grenade_data, damage_filters, grenade_filters)
    ud_df = ud_df[["Player", "UD", "UD Per Nade"]]
    flashes_df = flash_stats(flash_data, grenade_data, flash_filters, 
                             grenade_filters)
    flashes_df = flashes_df[["Player", "EF", "EF Per Throw"]]
    box_score = kills_df.merge(adr_df, how="outer")
    box_score = box_score[["Player", "K", "D", "A", "+/-", "KDR", "ADR", "HS%"]]
    box_score = box_score.merge(ud_df, how="outer")
    box_score = box_score.merge(flashes_df, how="outer")
    return box_score

player_box_score(data_df["Damages"], data_df["Flashes"], data_df["Grenades"], data_df["Kills"], data_df["Rounds"])

,Player,K,D,A,+/-,KDR,ADR,HS%,UD,UD Per Nade,EF,EF Per Throw
0,Hobbit,24,19,3,5,1.263158,88.000000,0.666667,191,6.586207,28,1.217391
1,b1t,23,18,1,5,1.277778,72.866667,0.478261,34,1.416667,40,1.333333
2,s1mple,22,14,4,8,1.571429,92.700000,0.272727,70,7.000000,55,1.527778
3,Ax1Le,18,16,2,2,1.125000,58.633333,0.555556,45,1.800000,35,1.093750
4,electronic,18,18,4,0,1.000000,70.133333,0.500000,111,4.269231,47,1.678571
5,interz,14,15,1,-1,0.933333,54.033333,0.500000,27,0.843750,36,1.500000
6,sh1ro,14,14,5,0,1.000000,56.833333,0.142857,105,6.562500,40,1.111111
7,Boombl4,12,16,2,-4,0.750000,57.100000,0.250000,277,8.147059,32,1.000000
8,nafany,11,21,3,-10,0.523810,54.000000,0.454545,176,7.040000,24,1.714286
9,Perfecto,10,16,2,-6,0.625000,34.133333,0.700000,48,1.371429,36,1.333333


# TEAM_BOX_SCORE_FUNCTION
The `function team_box_score` takes in damage data, flash data, grenade data, kill data, round data, and filters for each group of data, and return a team box score DataFrame containing statistics from each group of data.

In [15]:
def team_box_score(damage_data: pd.DataFrame,
                   flash_data: pd.DataFrame,
                   grenade_data: pd.DataFrame,
                   kill_data: pd.DataFrame,
                   round_data: pd.DataFrame,
                   round_data_json: List[Dict],
                   damage_filters: Dict[str, Union[List[bool], List[str]]] = {},
                   flash_filters: Dict[str, Union[List[bool], List[str]]] = {},
                   grenade_filters: Dict[str, Union[List[bool], List[str]]] = {},
                   kill_filters: Dict[str, Union[List[bool], List[str]]] = {},
                   death_filters: Dict[str, Union[List[bool], List[str]]] = {},
                   round_filters: Dict[str, Union[List[bool], List[str]]] = {}
) -> pd.DataFrame:
    kills = calc_stats(kill_data.loc[kill_data["AttackerTeam"] != 
                                        kill_data["VictimTeam"]], kill_filters, 
                          ["AttackerTeam"], ["AttackerTeam"], [["size"]], 
                          ["Team", "K"])
    deaths = calc_stats(kill_data, death_filters, ["VictimTeam"], ["VictimTeam"], 
                        [["size"]], ["Team", "D"])
    assists = calc_stats(kill_data.loc[(kill_data["AttackerTeam"] != 
                                        kill_data["VictimTeam"]) & 
                                       (kill_data["AssistedFlash"] == False)],
                         kill_filters, ["AssisterTeam"], ["AssisterTeam"], 
                         [["size"]], ["Team", "A"])
    first_kills = calc_stats(kill_data.loc[(kill_data["AttackerTeam"] != 
                                            kill_data["VictimTeam"]) &
                                           (kill_data["IsFirstKill"] == True)],
                             kill_filters, ["AttackerTeam"], ["AttackerTeam"], 
                             [["size"]], ["Team", "FK"])
    headshot_pct = calc_stats(kill_data.loc[kill_data["AttackerTeam"] != 
                                            kill_data["VictimTeam"]], 
                              kill_filters, ["AttackerTeam"], ["IsHeadshot"], 
                              [["mean"]], ["Team", "HS%"])
    adr = calc_stats(damage_data.loc[damage_data["AttackerTeam"] != 
                                     damage_data["VictimTeam"]], 
                     damage_filters, ["AttackerTeam"],["HpDamageTaken"], 
                     [["sum"]], ["Team", "ADR"])
    adr["ADR"] = adr["ADR"] / len(calc_stats(round_data, round_filters, [], [], 
                                             [], round_data.columns))
    util_dmg = calc_stats(damage_data.loc[(damage_data["AttackerTeam"] != 
                                           damage_data["VictimTeam"]) & 
                                          (damage_data["Weapon"].isin([ 
                                               "HE Grenade", 
                                               "Incendiary Grenade", 
                                               "Molotov"
                                          ]))], damage_filters, ["AttackerTeam"], 
                          ["HpDamage"], [["sum"]], ["Team", "UD"])
    nades_thrown = calc_stats(grenade_data.loc[grenade_data["GrenadeType"].isin([
                                                   "HE Grenade", 
                                                   "Incendiary Grenade", 
                                                   "Molotov"
                                               ])], grenade_filters, 
                              ["PlayerTeam"], ["PlayerTeam"], [["size"]], 
                              ["Team", "Nades Thrown"])   
    enemy_flashes = calc_stats(flash_data.loc[flash_data["AttackerTeam"] != 
                                              flash_data["PlayerTeam"]], 
                               flash_filters, ["AttackerTeam"], ["AttackerTeam"], 
                               [["size"]], ["Team", "EF"])
    flashes_thrown = calc_stats(grenade_data.loc[grenade_data["GrenadeType"] == 
                                                 "Flashbang"], flash_filters, 
                                ["PlayerTeam"], ["PlayerTeam"], [["size"]], 
                                ["Team", "Flashes Thrown"])
    econ_df = econ_stats(round_data, round_data_json)
    team_one = round_data_json[0]["TTeam"]
    box_score = kills.merge(deaths, how="outer")
    box_score = box_score.merge(assists, how="outer")
    box_score["+/-"] = box_score["K"] - box_score["D"]
    box_score = box_score.merge(first_kills, how="outer")
    box_score = box_score.merge(adr, how="outer")
    box_score = box_score.merge(headshot_pct, how="outer")
    box_score = box_score.merge(util_dmg, how="outer")
    box_score = box_score.merge(nades_thrown, how="outer")
    box_score["UD Per Nade"] = box_score["UD"]  / box_score["Nades Thrown"]
    box_score = box_score.merge(enemy_flashes, how="outer")
    box_score = box_score.merge(flashes_thrown, how="outer")
    box_score["EF Per Throw"] = box_score["EF"] / box_score["Flashes Thrown"]      
    if box_score.iloc[0]["Team"] == team_one:
        box_score["Avg EQ Value"] = [int(econ_df.iloc[0:2]["Avg EQ Value"].mean()), 
                                     int(econ_df.iloc[2:4]["Avg EQ Value"].mean())
                                    ] 
        box_score["Avg Spend"] = [int(econ_df.iloc[0:2]["Avg Spend"].mean()), 
                                  int(econ_df.iloc[2:4]["Avg Spend"].mean())] 
    else:
        box_score["Avg EQ Value"] = [int(econ_df.iloc[2:4]["Avg EQ Value"].mean()), 
                                     int(econ_df.iloc[0:2]["Avg EQ Value"].mean())
                                    ] 
        box_score["Avg Spend"] = [int(econ_df.iloc[2:4]["Avg Spend"].mean()), 
                                  int(econ_df.iloc[0:2]["Avg Spend"].mean())] 
    box_score = box_score.merge(win_breakdown(round_data), how="outer")
    box_score.rename(columns={"Total":"Score"}, inplace=True)
    score = box_score["Score"]
    box_score.drop("Score", axis=1, inplace=True)
    box_score.insert(1, "Score", score)
    box_score = box_score.transpose()
    box_score.columns = box_score.iloc[0]
    box_score.drop("Team", inplace=True)
    box_score = box_score.rename_axis(None, axis=1)
    return box_score

team_box_score(data_df["Damages"], data_df["Flashes"], data_df["Grenades"], data_df["Kills"], data_df["Rounds"], data["GameRounds"])

,Gambit,Natus Vincere
Score,16,14
K,81,85
D,85,82
A,14,13
+/-,-4,3
FK,17,13
ADR,311.5,326.933333
HS%,0.493827,0.423529
UD,544,540
Nades Thrown,127,129
